# Context


#### *What is India VIX*?
*The Volatility Index measures the market's anticipation of volatility in the near term. 
- During moments of market volatility, the market typically moves sharply up or down, and the VIX tends to climb.

- VIX falls as volatility falls. It is not the same as a price index like the NIFTY. 
- The price index is calculated by taking the price movement of the underlying equities into account.


#### *Scope of analysis*?
We will focusing on movment Nifty impacted by IndiaVix rather than focusing on how it is being impacted by option price.

# What questions are we trying to answer?


- Understand relationship between Nifty price behaviour with volatilty index
- Can we categories volatilty behaviours in nifity according to IndiaVix for better decision making
- Can we have guidlines on volatility of nifty in upcomming days by analysing historical data of IndiaVix


# Our approach to get these answers?
Analyse data in two time frames by analysing
1. Analysing last 4 month data as most recent behaviour.
2. Analysis of each financial year and how behaviour changed over the years.
3. Analysis of data from inception to create a base of all time.

---
---
---
---

## Tasks
- Get a reliable OHLC data source of Nifty50 and IndiaVIX data that auto-updates End of the day
- Join IndiaVix and Nifty50 data as df.
- 

---
---
---
---

# Sourcing Data

For sourcing we are using a library **jugaad-data**
it's important to fetch data from such library as it is updated every working day! 

In [15]:
from nsepython import *

In [26]:
import nsepy
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import time

In [27]:
# Define the start and end dates for the data
end_date = date(2023,3,31)
start_date = date(2008,1,1)
print(end_date)
print(start_date)

2023-03-31
2008-01-01


In [28]:
# Download the data
nifty50_data = nsepy.get_history('NIFTY 50', start=start_date, end=end_date, index=True)

# Print the data
print(nifty50_data)

Empty DataFrame
Columns: [Open, High, Low, Close, Volume, Turnover]
Index: []
